# [설명] 1. 환경 준비 · OpenAI 연결 (dotenv)
이 셀은 `.env`에서 API 키를 읽어 **OpenAI 클라이언트**를 준비합니다.  
- 왜? 교육 환경에서 키를 코드에 하드코딩하지 않고 안전하게 관리하기 위함  
- 무엇을 하나? `.env` 로드 → 키 검증 → 간단 헬퍼 `chat_local()` 정의  
- 기대 관찰: "✅ OpenAI 준비 완료" 출력  
- 흔한 이슈: `.env` 경로가 달라 못 읽는 경우 → 노트북 실행 경로에 `.env` 두기


In [ ]:
# 1. 환경 준비 · OpenAI 연결 (dotenv)
import os
from dotenv import load_dotenv
from openai import OpenAI

# .env 로드 (현재 작업 디렉터리의 .env 파일을 읽습니다)
_ = load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "OPENAI_API_KEY가 없습니다. 프로젝트 루트에 .env 파일을 만들고\n"
        "OPENAI_API_KEY=sk-... 를 넣은 뒤, 커널을 재시작하고 다시 실행하세요."
    )

# OpenAI 공식 엔드포인트 사용. base_url 지정 불필요.
client = OpenAI(api_key=api_key)

# 교육용 기본 모델(필요하면 바꾸세요: gpt-4o, gpt-4o-mini, o3-mini 등)
DEFAULT_MODEL = "gpt-4o-mini"

def chat_local(prompt: str, model: str | None = None, temperature: float = 0.2) -> str:
    """
    간단 래퍼:
    - 입력 프롬프트 한 개를 주면 한 문단 정도의 답을 반환합니다.
    - 나머지 셀에서 LM Studio → OpenAI 전환 시 함수명만 동일하게 유지해도 되도록 통일했습니다.
    """
    model = model or DEFAULT_MODEL
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    return (resp.choices[0].message.content or "").strip()

print("OpenAI 준비 완료")


In [ ]:
# === LM Studio 로컬 모델 연결 ===
# 사전 준비:
# 1) LM Studio 실행 → 모델 로드 → Server 탭에서 HTTP 서버 Start
# 2) 프로젝트 루트에 .env 파일 생성:
#    LM_STUDIO_BASE_URL=http://localhost:1234/v1
#    LM_STUDIO_API_KEY=lm-studio         # 반드시 ASCII 문자만 (한글/이모지 X)
#    LM_STUDIO_MODEL=YourLocalModelName  # (선택) LM Studio Server에 표시된 모델 id

import os
from dotenv import load_dotenv
from openai import OpenAI

# .env 로드
_ = load_dotenv()

# 기본값/전처리 (공백·따옴표 제거)
BASE_URL = (os.getenv("LM_STUDIO_BASE_URL", "http://localhost:1234/v1") or "").strip().strip('"').strip("'")
RAW_KEY  = (os.getenv("LM_STUDIO_API_KEY", "lm-studio") or "").strip().strip('"').strip("'")
DEFAULT_MODEL = (os.getenv("LM_STUDIO_MODEL") or "").strip().strip('"').strip("'") or None

# ASCII 강제: 헤더 인코딩 오류(UnicodeEncodeError) 방지
API_KEY = RAW_KEY if RAW_KEY.isascii() else "lm-studio"

# OpenAI 호환 인터페이스로 LM Studio에 연결
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

# 모델 목록 확인 및 기본 모델 선택
try:
    models = client.models.list()
    model_ids = [m.id for m in models.data]
except Exception as e:
    raise RuntimeError(
        f"LM Studio 연결 실패: {e}\n"
        f"- LM Studio의 Server 탭에서 HTTP 서버가 켜져 있는지 확인하세요.\n"
        f"- BASE_URL={BASE_URL}\n"
        f"- LM_STUDIO_API_KEY는 ASCII만 사용하세요(한글/이모지 X)."
    )

if not DEFAULT_MODEL:
    if model_ids:
        DEFAULT_MODEL = model_ids[0]
    else:
        raise RuntimeError(
            "실행 중인 모델을 찾지 못했습니다. LM Studio에서 모델을 Start 하거나, "
            "'.env'에 LM_STUDIO_MODEL=... 을 지정하세요."
        )

def chat_local(prompt: str, model: str | None = None, temperature: float = 0.2) -> str:
    """
    LM Studio 서버로 Chat Completions 호출 (OpenAI 호환)
    - 노트북의 다른 셀에서 동일한 함수명(chat_local)을 사용하므로 drop-in 교체가 쉽습니다.
    """
    model = model or DEFAULT_MODEL
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    return (resp.choices[0].message.content or "").strip()

print("✅ LM Studio 준비 완료")
print(f"- base_url: {BASE_URL}")
print(f"- api_key(ascii?): {API_KEY.isascii()}")
print(f"- model:    {DEFAULT_MODEL}")

# (선택) 스모크 테스트
print(chat_local("한 줄로 인사해줘"))


# 2. LangGraph 기본 임포트 · 상태 스키마 정의
HITL 그래프에 사용할 **상태(State)**를 정의합니다.  
- 왜? LangGraph는 “딕셔너리 상태를 노드들이 갱신해 나가는” 구조라, 키 스키마가 명확해야 디버깅이 쉽습니다.  
- 무엇을 하나? `S` 타입에 입력/중간/최종 결과 필드를 선언  

In [ ]:
# 2. LangGraph 기본 임포트 · 상태 스키마 정의
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

class S(TypedDict, total=False):
    # ▶ 입력
    question: str           # 사용자가 던진 질문/주제
    notes: str              # (선택) 참고 메모(톤, 범위, 금지어 등)

    # ▶ 중간 산출물
    draft: str              # LLM이 작성한 초안
    review_comment: str     # 사람이 입력한 승인/수정 코멘트
    approved: bool          # 승인 여부(True면 finalize로 진행)

    # ▶ 최종 산출물
    final: str              # 승인되거나 수정 반영된 최종 답변

print("상태 스키마 준비")

# 3. 노드 정의: draft → review(HITL) → revise → finalize
각 노드는 상태 일부를 읽고, 일부를 **반환(갱신)**합니다.  
- `node_draft`: 질문/메모 기반으로 초안을 생성  
- `node_review`: **사람 개입(HITL)** — 콘솔에서 승인(Enter) 또는 수정 코멘트 입력  
- `node_revise`: 코멘트를 반영해 초안을 재작성  
- `node_finalize`: 승인된 초안을 최종본으로 다듬기(말끝/중복 정리)


In [ ]:
# 3. 노드 정의: draft → review(HITL) → revise → finalize
def node_draft(state: S) -> S:
    """
    초안 생성 노드
    - 입력: question, notes
    - 출력: draft, approved(False로 초기화)
    """
    q = (state.get("question") or "").strip()
    notes = state.get("notes", "")
    prompt = f"""다음 질문에 대해 4~6줄 요약 초안을 작성해줘.
- 질문: {q}
- 메모(참고): {notes}
- 요구사항: 명확하고 간결하게. 마지막에 '검토 요청' 문구 1줄 포함."""
    draft = chat_local(prompt, temperature=0.3)
    return {"draft": draft, "approved": False}

def node_review(state: S) -> S:
    """
    HITL(사람 개입) 리뷰 노드
    - 동작: 초안을 출력하고 사용자의 입력을 받음
      * 빈 입력(Enter): 승인
      * 문자열 입력: 수정 코멘트로 간주
    - 출력: review_comment, approved
    """
    print("\n" + "="*60)
    print("🔎 초안(DRAFT)")
    print("-"*60)
    print(state.get("draft", "(초안 없음)"))
    print("-"*60)
    print("👤 리뷰 방법")
    print(" - 승인하려면: 그냥 Enter (빈 입력)")
    print(" - 수정 코멘트를 남기려면: 내용을 입력 후 Enter")
    print("="*60)

    user_input = input("리뷰 코멘트 입력(빈 입력 = 승인): ").strip()
    if user_input == "":
        return {"review_comment": "승인", "approved": True}
    else:
        return {"review_comment": user_input, "approved": False}

def node_revise(state: S) -> S:
    """
    재작성 노드
    - 입력: draft, review_comment
    - 출력: draft(덮어쓰기), approved(False 유지 → 다시 review로)
    """
    draft = state.get("draft", "")
    comment = state.get("review_comment", "")
    prompt = f"""아래 초안을 사람의 코멘트를 반영해 더 나은 버전으로 수정해줘.
- 초안:
{draft}

- 사람 코멘트:
{comment}

- 요구사항:
1) 핵심을 유지하되, 코멘트의 지적사항을 반영
2) 4~6줄 분량으로 간결하게
3) 마지막 줄에 '검토 요청' 문구 유지
"""
    revised = chat_local(prompt, temperature=0.3)
    return {"draft": revised, "approved": False}

def node_finalize(state: S) -> S:
    """
    최종 정리 노드
    - 입력: 승인된 draft
    - 출력: final (검토 요청 문구 제거, 문체 정돈)
    """
    draft = state.get("draft", "")
    prompt = f"""아래 초안을 최종본으로 정리해줘.
- 초안:
{draft}

- 요구사항:
1) 불필요한 중복 제거
2) 문장 간 자연스러운 연결
3) 마무리 문구에서 '검토 요청' 문구 제거
4) 4~6줄
"""
    final = chat_local(prompt, temperature=0.2)
    return {"final": final}

print("노드 정의 완료")


# 4. 분기 함수 & 그래프 구성
- 분기 함수 `route_after_review`:
  - 승인되면 `"approved"` 라벨 반환 → `finalize`로  
  - 미승인이면 `"revise"` 라벨 반환 → `revise`로
- 그래프 엣지:
  - `START → draft → review → (approved? finalize : revise → review)` → `END`
- 체크포인트:
  - `MemorySaver`로 같은 `thread_id`에서 재실행/스트리밍 시 추적 가능


In [ ]:
# 4. 분기 함수 & 그래프 구성
def route_after_review(state: S) -> str:
    """리뷰 결과에 따라 다음 노드 라벨을 결정"""
    return "approved" if state.get("approved") else "revise"

builder = StateGraph(S)

# 노드 등록
builder.add_node("draft", node_draft)
builder.add_node("review", node_review)
builder.add_node("revise", node_revise)
builder.add_node("finalize", node_finalize)

# 엣지 구성
builder.add_edge(START, "draft")
builder.add_edge("draft", "review")
builder.add_conditional_edges("review", route_after_review, {
    "approved": "finalize",
    "revise": "revise",
})
builder.add_edge("revise", "review")   # 재작성 후 다시 리뷰로 루프
builder.add_edge("finalize", END)

# 체크포인트(선택)
checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)

print("그래프 컴파일 완료")


# 5. 실행 데모 1 — 한 번의 승인 흐름
- 입력: question + notes  
- 동작: `draft`가 생성되고, `review`에서 **Enter**(승인) → `finalize`로 종료  
- 기대 관찰: 마지막에 "=== 최종본(FINAL) ===" 아래 정리된 답변 출력


In [ ]:
# 5. 실행 데모 1 — 한 번의 승인 흐름
thread = {"configurable": {"thread_id": "hitl-demo-1"}}

payload = {
    "question": "LangGraph에서 사람 개입(HITL)을 어떻게 설계하면 좋은가?",
    "notes": "입문자 대상. 리뷰-재작성 루프를 직관적으로 보여줄 것."
}

# (선택) 중간 상태 관찰: stream_mode="values"
for u in app.stream(payload, thread, stream_mode="values"):
    print(u)

result = app.invoke(payload, thread)
print("\n\n=== 최종본(FINAL) ===\n")
print(result.get("final", "(최종본 없음)"))


# 6. 실행 데모 2 — 수정 코멘트 후 재승인 흐름
- 첫 리뷰에서 **수정 코멘트**를 입력 → `revise`에서 반영 → `review`로 복귀  
- 두 번째 리뷰에서 **Enter(승인)** → `finalize`로 종료  
- 기대 관찰: 코멘트가 반영된 초안이 두 번째 리뷰에서 확인됨


In [ ]:
# 6. 실행 데모 2 — 수정 코멘트 후 재승인 흐름
thread2 = {"configurable": {"thread_id": "hitl-demo-2"}}

result2 = app.invoke({
    "question": "RAG 파이프라인에서 사람 검토 지점은 어디에 배치하는 게 효과적인가?",
    "notes": "품질, 리스크 높은 섹션에 리뷰 스텝 배치. 승인 후에만 다음 단계 진행."
}, thread2)

print("\n\n=== 최종본(FINAL) ===\n")
print(result2.get("final", "(최종본 없음)"))


# 7. 결과 저장 — 문서화/리뷰 기록
- 왜? 교육 후 산출물을 파일로 보관하여 품질 비교/리뷰 회고에 활용  
- 무엇? `outputs/` 폴더에 타임스탬프 파일로 최종본 저장  
- 기대 관찰: "✅ 저장 완료: outputs/..." 경로 출력


In [ ]:
# 7. 결과 저장 — 문서화/리뷰 기록
import time, pathlib

def save_final(state: S, filename_prefix: str = "hitl_final"):
    out_dir = pathlib.Path("outputs")
    out_dir.mkdir(exist_ok=True)
    ts = time.strftime("%Y%m%d-%H%M%S")
    path = out_dir / f"{filename_prefix}_{ts}.md"
    with open(path, "w", encoding="utf-8") as f:
        f.write("# HITL 최종본\n\n")
        f.write("## 질문\n")
        f.write((state.get("question") or "") + "\n\n")
        f.write("## 최종 답변\n")
        f.write(state.get("final", "(없음)") + "\n")
    print(f"저장 완료: {path}")
    return path

# 데모1 결과 저장(원한다면 데모2도 저장 가능)
_ = save_final(result, filename_prefix="hitl_demo1")


# HITL 리뷰를 GUI(Gradio)로 바꾸기
목표: 콘솔 `input()` 대신 웹 UI에서 **승인/수정**을 버튼과 코멘트 입력으로 처리한다.

구성:
- 전역 `queue.Queue()`로 LangGraph(백엔드) ↔ Gradio(프론트) 통신
- Gradio에서 "승인" 클릭 → `{"approved": True, "review_comment": "승인"}`
- Gradio에서 "수정 요청" 제출 → `{"approved": False, "review_comment": ...}`
- `node_review_gui` 노드는 큐에서 **사람의 입력이 올 때까지 대기**한 뒤 상태 갱신


In [ ]:
# [포트/주소 명시 출력, Gradio 4.x 호환
# - prevent_thread_lock=True 로 노트북 블로킹 없이 실행
# - 로컬/공유 URL을 직접 print
# - 포트 미지정 시 7860, 사용 중이면 자동으로 다음 포트 검색

import gradio as gr
import queue, socket

# 백엔드 ↔ 프론트 공유 전역
CURRENT_DRAFT = {"text": "", "version": 0}
REVIEW_QUEUE: "queue.Queue[dict]" = queue.Queue()

def pull_draft(curr_ver: int):
    """UI 폴링: 버전이 바뀌면 텍스트/버전을 갱신, 아니면 유지"""
    if CURRENT_DRAFT["version"] != curr_ver:
        return CURRENT_DRAFT["text"], CURRENT_DRAFT["version"]
    return gr.update(), curr_ver

def approve_fn(draft_text: str):
    REVIEW_QUEUE.put({"approved": True, "review_comment": "승인"})
    return f"(승인됨)\n\n{draft_text}"

def request_fn(draft_text: str, comment_text: str):
    c = (comment_text or "").strip() or "(사유 없음) 수정 요청"
    REVIEW_QUEUE.put({"approved": False, "review_comment": c})
    return f"(수정 요청됨)\n\n사유: {c}\n\n{draft_text}"

def _find_free_port(start=7860, limit=20):
    for p in range(start, start+limit):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("127.0.0.1", p))
                return p
            except OSError:
                continue
    return None

def _launch_auto_ui(port: int | None = None, share: bool = False):
    with gr.Blocks() as demo:
        gr.Markdown("## 🔎 HITL 리뷰 (자동 채움)")
        ver_state = gr.State(0)  # 현재 표시 중인 버전
        draft_box = gr.Textbox(label="초안(DRAFT, 자동 반영)", value="", lines=12)
        comment   = gr.Textbox(label="수정 코멘트(비우고 '승인'을 누르면 승인)", lines=3)
        with gr.Row():
            approve_btn = gr.Button("✅ 승인", variant="primary")
            request_btn = gr.Button("✍️ 수정 요청")

        approve_btn.click(approve_fn, inputs=[draft_box], outputs=[draft_box])
        request_btn.click(request_fn, inputs=[draft_box, comment], outputs=[draft_box])

        # 폴링: 신형 load(every=) 우선, 실패 시 Timer.tick 폴백
        try:
            demo.load(pull_draft, inputs=[ver_state], outputs=[draft_box, ver_state], every=0.8)
        except Exception:
            try:
                timer = gr.Timer(0.8)
                timer.tick(pull_draft, inputs=[ver_state], outputs=[draft_box, ver_state])
            except Exception:
                demo.load(pull_draft, inputs=[ver_state], outputs=[draft_box, ver_state])

    if port is None:
        port = _find_free_port() or 7860

    # queue()는 인자 없이 호출(Gradio 4.x)
    try:
        demo.queue()
    except TypeError:
        pass

    server = demo.launch(
        server_name="127.0.0.1",
        server_port=port,
        share=share,
        prevent_thread_lock=True,
        show_error=True,
    )

    # 주소 출력(Gradio 4.x는 LaunchResult에 local_url/share_url 제공)
    local_url = getattr(server, "local_url", f"http://127.0.0.1:{port}")
    share_url = getattr(server, "share_url", None) if share else None
    print("✅ Gradio 리뷰 UI(자동 채움) 실행됨")
    print(f"🔗 Local: {local_url}")
    if share_url:
        print(f"🌍 Share: {share_url}")

    return server

_AUTO_REVIEW_SERVER = _launch_auto_ui(port=None, share=False) # share=True로 변경 시 외부에서도 페이지 확인 가능(로컬이 아닌 위치에서)


In [ ]:
# 리뷰 노드(자동 채움)로 교체하고 그래프 재컴파일
# - node_review_gui_auto: draft를 CURRENT_DRAFT에 기록 → 큐에서 승인/수정 신호를 기다림
# - 기존 review 노드를 이 함수로 교체

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

def node_review_gui_auto(state: "S") -> "S":
    """
    자동 채움 리뷰 노드:
    1) 현재 draft를 글로벌 CURRENT_DRAFT에 기록(+버전 증가)
    2) 사용자가 Gradio에서 '승인/수정' 버튼을 누를 때까지 큐에서 대기
    """
    draft = state.get("draft", "")
    # 초안 전달(버전 증가 → UI Timer가 감지해 자동 반영)
    CURRENT_DRAFT["text"] = draft
    CURRENT_DRAFT["version"] += 1
    print("🔔 리뷰 UI로 초안이 전송되었습니다. (자동 채움)")

    # 승인/수정 신호 수신까지 블로킹
    review = REVIEW_QUEUE.get()  # {"approved": bool, "review_comment": str}
    return {"approved": review["approved"], "review_comment": review["review_comment"]}

# 그래프 재구성 (기존 node_draft/node_revise/node_finalize/S 사용)
builder = StateGraph(S)
builder.add_node("draft",    node_draft)           # 기존 초안 생성 노드
builder.add_node("review",   node_review_gui_auto) # 자동 채움 리뷰 노드로 교체
builder.add_node("revise",   node_revise)          # 코멘트 반영 재작성
builder.add_node("finalize", node_finalize)        # 최종 정리

builder.add_edge(START, "draft")
builder.add_edge("draft", "review")
builder.add_conditional_edges("review",
                              lambda s: "approved" if s.get("approved") else "revise",
                              {"approved": "finalize", "revise": "revise"})
builder.add_edge("revise", "review")
builder.add_edge("finalize", END)

app_auto = builder.compile(checkpointer=MemorySaver())
print("✅ 자동 채움 리뷰 버전 그래프 컴파일 완료")


In [ ]:
# 실행 데모
# - 이 셀을 실행하면 초안이 생성되고, Gradio UI 텍스트박스에 자동으로 채워진다.
# - UI에서 '✅ 승인' 또는 '✍️ 수정 요청'을 누르면 그래프가 다음 단계로 진행되어 최종본을 출력한다.

thread_auto = {"configurable": {"thread_id": "hitl-auto-1"}}
res_auto = app_auto.invoke({
    "question": "HITL(사람 개입) 자동 채움 UI를 쓰면 어떤 교육적 장점이 있나?",
    "notes": "복사/붙여넣기 없이 승인/수정 UX를 보여줄 것."
}, thread_auto)

print("\n=== 최종본(FINAL, 자동 채움) ===\n")
print(res_auto.get("final", "(없음)"))
